Brenda Woodard
Data Mining Assignment 2

In this assignment you are to use Decision Trees & Naive Bayes for classification. In addition, you are to explore your models to determine which feature vector attributes you can remove without "significantly hurting" classification importance. There is no single "right" answer for this question. Also, "significantly hurting" is a judgment call, please explain what you mean by that & provide numeric values. You should start with the full dataset & look at the decision tree feature_importances_ & naive bayes permutation_importance to guide which attributes to remove. You will then want to iteratively remove attributes, possibly one at a time or several at a time. Again, there is no single "right way" to do this.

The goal is to predict whether it will rain tomorrow given the days weather statistics. The last column, "Rain Tomorrow" is the classification value for I have cleaned the data set for you. I removed a few attributes that were almost all zeros, & replaced missing values with the mean value for that column. Also, strings have been transformed into numeric values so the data set will work well for both decision trees and naive bayes. 

Turn In:

1) A report that explains which attributes you have eliminated, why those, & what effect this has on the quality of predictions compared to using the full dataset. Feel free to include a table of feature_importances & permutation_importances that show how & why you reduced the set of attributes.

2) You code for creating the decision tree & naive bayes models & getting the feature/permuttion importances. Your code does not need to cover your whole experimental space, but should provide enough information to show how you made at least one decision about which attribute (column) to eliminate.

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.naive_bayes import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.inspection import permutation_importance

dat = pd.read_csv('assignment2_cleanInfile.csv')
dat.head(20)

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2,13.4,22.9,0.600000,13,44.0,13,14,20.000000,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,0.000000,0
1,2,7.4,25.1,0.000000,14,44.0,6,15,4.000000,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,0.000000,0
2,2,12.9,25.7,0.000000,15,46.0,13,15,19.000000,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,0.000000,0
3,2,9.2,28.0,0.000000,4,24.0,9,0,11.000000,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,0.000000,0
4,2,17.5,32.3,1.000000,13,41.0,1,7,7.000000,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0.000000,0
5,2,14.6,29.7,0.200000,14,56.0,13,13,19.000000,24.0,55.0,23.0,1009.2,1005.4,4.437189,4.503167,20.6,28.9,0.000000,0
6,2,14.3,25.0,0.000000,13,50.0,12,13,20.000000,24.0,49.0,19.0,1009.6,1008.2,1.000000,4.503167,18.1,24.6,0.000000,0
7,2,7.7,26.7,0.000000,13,35.0,10,13,6.000000,17.0,48.0,19.0,1013.4,1010.1,4.437189,4.503167,16.3,25.5,0.000000,0
8,2,9.7,31.9,0.000000,6,80.0,9,7,7.000000,28.0,42.0,9.0,1008.9,1003.6,4.437189,4.503167,18.3,30.2,0.000000,1
9,2,13.1,30.1,1.400000,13,28.0,8,10,15.000000,11.0,58.0,27.0,1007.0,1005.7,4.437189,4.503167,20.1,28.2,1.000000,0


In [3]:
Y = dat['RainTomorrow']
data = dat.drop('RainTomorrow', axis=1)

x_train, x_test, y_train, y_test  = train_test_split(data, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree = tree.DecisionTreeClassifier(criterion="gini")
dtree = dtree.fit(x_train,y_train)
y_predicted = dtree.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix(y_test, y_predicted))
importance = dtree.feature_importances_
print("decision tree dtree feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, data.columns[i], v) )
    # print('Feature: %0d, Score: %.5f' % (i,v))

print("\n\nGausianNB:")
model = GaussianNB()
model.fit(x_train,y_train)
gausianNB_predicted = model.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix( y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)

x_test =         Location  MinTemp  MaxTemp  Rainfall  WindGustDir  WindGustSpeed  \
27951         34     19.9     24.9       0.4            9           26.0   
57687          6     -2.6     13.5       0.2           12           30.0   
121721        36     10.6     16.8       0.0            7           50.0   
116392        32     11.6     25.6       0.0            2           61.0   
58781          6     10.7     37.3       0.0           12           57.0   
...          ...      ...      ...       ...          ...            ...   
34105         38      9.0     19.7       3.4           14           31.0   
85592          8     23.0     32.6      16.8            1           31.0   
137829        13     24.3     32.7      23.2            1           57.0   
61461         35     17.6     32.8       0.0           10           61.0   
92981         39     27.7     36.0       0.0           15           39.0   

        WindDir9am  WindDir3pm  WindSpeed9am  WindSpeed3pm  Humidity9am  \
279

In [7]:
# now drop all attributes that scored less than 0.05 from decision tree
print("\n\nDropping attributes with Decision Tree Score <0.05")
df2 = data.drop(['Location', 'MaxTemp', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 
                'Pressure9am', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday'], axis = 1)
print(df2)

x_train, x_test, y_train, y_test  = train_test_split(df2, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree2 = tree.DecisionTreeClassifier(criterion="gini")
dtree2 = dtree2.fit(x_train,y_train)
y_predicted = dtree2.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree2.feature_importances_
print("decision tree dtree2 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df2.columns[i], v) )

print("\n\nGausianNB:")
model2 = GaussianNB()
model2.fit(x_train,y_train)
gausianNB_predicted = model2.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model2, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Dropping attributes with Decision Tree Score <0.05
        MinTemp  Rainfall  WindGustSpeed  Humidity9am  Humidity3pm  \
0          13.4       0.6           44.0         71.0         22.0   
1           7.4       0.0           44.0         44.0         25.0   
2          12.9       0.0           46.0         38.0         30.0   
3           9.2       0.0           24.0         45.0         16.0   
4          17.5       1.0           41.0         82.0         33.0   
...         ...       ...            ...          ...          ...   
142188      3.5       0.0           31.0         59.0         27.0   
142189      2.8       0.0           31.0         51.0         24.0   
142190      3.6       0.0           22.0         56.0         21.0   
142191      5.4       0.0           37.0         53.0         24.0   
142192      7.8       0.0           28.0         51.0         24.0   

        Pressure3pm  
0            1007.1  
1            1007.8  
2            1008.7  
3            1012.

In [11]:
# now drop all attributes left that also scored less than 1 from Naive Bayes
print("\n\nAlso dropping attributes with Naive Bayes Score <1")
df3 = df2.drop('Humidity9am', axis=1)
print(df3)

x_train, x_test, y_train, y_test  = train_test_split(df3, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree3 = tree.DecisionTreeClassifier(criterion="gini")
dtree3 = dtree3.fit(x_train,y_train)
y_predicted = dtree3.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree3.feature_importances_
print("decision tree dtree3 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df3.columns[i], v) )

print("\n\nGausianNB:")
model3 = GaussianNB()
model3.fit(x_train,y_train)
gausianNB_predicted = model3.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model3, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Also dropping attributes with Naive Bayes Score <1
        MinTemp  Rainfall  WindGustSpeed  Humidity3pm  Pressure3pm
0          13.4       0.6           44.0         22.0       1007.1
1           7.4       0.0           44.0         25.0       1007.8
2          12.9       0.0           46.0         30.0       1008.7
3           9.2       0.0           24.0         16.0       1012.8
4          17.5       1.0           41.0         33.0       1006.0
...         ...       ...            ...          ...          ...
142188      3.5       0.0           31.0         27.0       1021.2
142189      2.8       0.0           31.0         24.0       1020.3
142190      3.6       0.0           22.0         21.0       1019.1
142191      5.4       0.0           37.0         24.0       1016.8
142192      7.8       0.0           28.0         24.0       1016.5

[142193 rows x 5 columns]
x_test =         MinTemp  Rainfall  WindGustSpeed  Humidity3pm  Pressure3pm
27951      19.9       0.4           26.0

In [14]:
# Since dropping Humidity9am made the accuracy worse, drop the MinTemp column instead
print("\n\nSince dropping Humidity9am made the accuracy worse, drop the MinTemp column instead")
df4 = df2.drop('MinTemp', axis=1)
print(df4)

x_train, x_test, y_train, y_test  = train_test_split(df4, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree4 = tree.DecisionTreeClassifier(criterion="gini")
dtree4 = dtree4.fit(x_train,y_train)
y_predicted = dtree4.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree4.feature_importances_
print("decision tree dtree4 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df4.columns[i], v) )

print("\n\nGausianNB:")
model4 = GaussianNB()
model4.fit(x_train,y_train)
gausianNB_predicted = model4.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model3, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Also dropping attributes with Naive Bayes Score <1
        Rainfall  WindGustSpeed  Humidity9am  Humidity3pm  Pressure3pm
0            0.6           44.0         71.0         22.0       1007.1
1            0.0           44.0         44.0         25.0       1007.8
2            0.0           46.0         38.0         30.0       1008.7
3            0.0           24.0         45.0         16.0       1012.8
4            1.0           41.0         82.0         33.0       1006.0
...          ...            ...          ...          ...          ...
142188       0.0           31.0         59.0         27.0       1021.2
142189       0.0           31.0         51.0         24.0       1020.3
142190       0.0           22.0         56.0         21.0       1019.1
142191       0.0           37.0         53.0         24.0       1016.8
142192       0.0           28.0         51.0         24.0       1016.5

[142193 rows x 5 columns]
x_test =         Rainfall  WindGustSpeed  Humidity9am  Humidity3pm  

In [15]:
# Dropping the MinTemp column increased accuracy.
# Drop WindGustSpeed since it scores negative from Naive Bayes and poorly from decision tree as well
print("\n\nDrop WindGustSpeed since it scores negative from Naive Bayes and poorly from decision tree as well")
df5 = df4.drop('WindGustSpeed', axis=1)
print(df5)

x_train, x_test, y_train, y_test  = train_test_split(df5, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree5 = tree.DecisionTreeClassifier(criterion="gini")
dtree5 = dtree5.fit(x_train,y_train)
y_predicted = dtree5.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree5.feature_importances_
print("decision tree dtree5 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df5.columns[i], v) )

print("\n\nGausianNB:")
model5 = GaussianNB()
model5.fit(x_train,y_train)
gausianNB_predicted = model5.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model5, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Also dropping attributes with Naive Bayes Score <1
        Rainfall  Humidity9am  Humidity3pm  Pressure3pm
0            0.6         71.0         22.0       1007.1
1            0.0         44.0         25.0       1007.8
2            0.0         38.0         30.0       1008.7
3            0.0         45.0         16.0       1012.8
4            1.0         82.0         33.0       1006.0
...          ...          ...          ...          ...
142188       0.0         59.0         27.0       1021.2
142189       0.0         51.0         24.0       1020.3
142190       0.0         56.0         21.0       1019.1
142191       0.0         53.0         24.0       1016.8
142192       0.0         51.0         24.0       1016.5

[142193 rows x 4 columns]
x_test =         Rainfall  Humidity9am  Humidity3pm  Pressure3pm
27951        0.4         81.0         68.0  1015.700000
57687        0.2         63.0         42.0  1025.100000
121721       0.0         77.0         61.0  1015.258204
116392       0.

In [16]:
# Dropping the WindGustSpeed column decreased accuracy slightly.
# Drop Humidity9am since it scores poorly from both the Naive Bayes and decision tree
print("\n\nDrop Humidity9am since it scores poorly from both the Naive Bayes and decision tree")
df6 = df5.drop('Humidity9am', axis=1)
print(df6)

x_train, x_test, y_train, y_test  = train_test_split(df6, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree6 = tree.DecisionTreeClassifier(criterion="gini")
dtree6 = dtree6.fit(x_train,y_train)
y_predicted = dtree6.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree6.feature_importances_
print("decision tree dtree6 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df6.columns[i], v) )

print("\n\nGausianNB:")
model6 = GaussianNB()
model6.fit(x_train,y_train)
gausianNB_predicted = model6.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model6, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Drop Humidity9am since it scores poorly from both the Naive Bayes and decision tree
        Rainfall  Humidity3pm  Pressure3pm
0            0.6         22.0       1007.1
1            0.0         25.0       1007.8
2            0.0         30.0       1008.7
3            0.0         16.0       1012.8
4            1.0         33.0       1006.0
...          ...          ...          ...
142188       0.0         27.0       1021.2
142189       0.0         24.0       1020.3
142190       0.0         21.0       1019.1
142191       0.0         24.0       1016.8
142192       0.0         24.0       1016.5

[142193 rows x 3 columns]
x_test =         Rainfall  Humidity3pm  Pressure3pm
27951        0.4         68.0  1015.700000
57687        0.2         42.0  1025.100000
121721       0.0         61.0  1015.258204
116392       0.0         22.0  1011.400000
58781        0.0          6.0  1011.500000
...          ...          ...          ...
34105        3.4         46.0  1019.100000
85592       16.8  

In [17]:
# Dropping the Humidity9am column decreased accuracy slightly.
# Drop Rainfall since it scores poorly from both the Naive Bayes and decision tree
print("\n\nDrop Rainfall since it scores poorly from both the Naive Bayes and decision tree")
df7 = df6.drop('Rainfall', axis=1)
print(df7)

x_train, x_test, y_train, y_test  = train_test_split(df7, Y,  test_size=0.25, random_state=2)
print('x_test = ' + str(x_test) )
print('y_test = ' + str(y_test) )

print("\n\nDecision Tree:")
dtree7 = tree.DecisionTreeClassifier(criterion="gini")
dtree7 = dtree7.fit(x_train,y_train)
y_predicted = dtree7.predict(x_test)
print('DecisionTree confusion matrix:')
print(confusion_matrix( y_test, y_predicted))
importance = dtree7.feature_importances_
print("decision tree dtree7 feature importance:")
for i,v in enumerate(importance):
    print('Feature: %0d, FName: %15s, Score: %.5f' % (i, df7.columns[i], v) )

print("\n\nGausianNB:")
model7 = GaussianNB()
model7.fit(x_train,y_train)
gausianNB_predicted = model7.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix(y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
imps = permutation_importance(model7, x_test, y_test)
print("gaussianNB feature importance:")
print(imps.importances_mean)



Drop Rainfall since it scores poorly from both the Naive Bayes and decision tree
        Humidity3pm  Pressure3pm
0              22.0       1007.1
1              25.0       1007.8
2              30.0       1008.7
3              16.0       1012.8
4              33.0       1006.0
...             ...          ...
142188         27.0       1021.2
142189         24.0       1020.3
142190         21.0       1019.1
142191         24.0       1016.8
142192         24.0       1016.5

[142193 rows x 2 columns]
x_test =         Humidity3pm  Pressure3pm
27951          68.0  1015.700000
57687          42.0  1025.100000
121721         61.0  1015.258204
116392         22.0  1011.400000
58781           6.0  1011.500000
...             ...          ...
34105          46.0  1019.100000
85592          63.0  1008.600000
137829         74.0  1006.800000
61461          34.0  1010.600000
92981          48.0   999.200000

[35549 rows x 2 columns]
y_test = 27951     0
57687     0
121721    0
116392    0
58781 

In [ ]:
# Dropping the Rainfall column gives the most accurate score yet.